## NPL Project Notebook

#### Libraries, Imports and Data

In [1]:
import csv                               # csv reader
import nltk
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn import metrics as Skmet #used for precision_recall_fscore_support()
from operator import itemgetter #used to unpack turples
import re
import math

#### Preprocessing and Data Loading

In [ ]:
def preProcess(text):
    # Should return a list of tokens
    text = text.lower() #normalize the text 
    text = re.sub(r"<.*>", " ", text) # speach mark removal.
    text = re.sub(r".\'", "" , text) # to prevent 'don't' becoming 'don' 't'
    tokenizer = RegexpTokenizer(r'\w+')
    preNormTemp = tokenizer.tokenize(text)

    lemmatizer = WordNetLemmatizer()
    stopWords = set(stopwords.words('english')) #stop word removal.
    
    
    for w in preNormTemp: #stop word removal and lemmatization
        if (w not in stopWords):
            temp.append(w)  #lemmatizer.lemmatize(w))
        if 
    temp = text.split(" ") 
    
    return preNormTemp

def loadData(path, Text=None):
    with open(path,encoding='utf8') as f:
        reader = csv.reader(f, delimiter='\t')
        count = 0
        for line in reader:
            if count == 0:
                count += 1
            else:
                (line, character, gender) = parseReview(line)
                rawText.append(line, character, gender);
                preprocessedData.append((preProcess(line), character, gender))
                count += 1
        
def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    
    for (line, chatacter, Label, Verified, Title) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text),preProcess(Title),Verified),Label))
    
    for (_, Text, Label, Verified, Title) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text),preProcess(Title),Verified),Label))

#### Training

In [4]:
stopWords = set(stopwords.words('english'))
print(stopWords)
    


{'ain', 'on', "she's", 'as', "wouldn't", "you're", 'each', 'ourselves', 'ours', "mustn't", 'does', 'not', 'i', 'what', 'then', 'have', 'during', 'until', 'about', 't', 'than', 'when', "hasn't", 'most', 'at', 'here', 'too', 'will', 'y', 'won', "weren't", 'itself', 'more', 're', 'you', 'by', 'before', 'can', 'over', "won't", 'because', 'them', 'd', 'in', 'these', 'did', 'under', 'very', 'only', 'she', 'has', 'out', 'who', 'such', 'down', 'doing', 'your', 'up', 'couldn', 'hers', 'we', "doesn't", 'aren', 'are', 'further', 'of', 'again', 'themselves', "you'd", 'few', 'other', 'ma', 'both', 'a', 'having', 'doesn', 'was', 'off', 'just', 'mustn', 'isn', 'any', "wasn't", 'haven', 'for', "you'll", "shouldn't", 'through', "don't", 'yours', 'now', 'our', 'shan', 'him', 'an', 'why', 'my', "aren't", 'while', "it's", 'himself', 'theirs', 'nor', 'mightn', 'myself', 'do', 'his', 'it', 'their', 'into', 'is', 'how', 'own', 'he', "should've", 'with', 'if', 'against', 'm', 'some', 'wouldn', 'or', 'its', "d

#### Cross Validation and Testing

In [ ]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    #print(dataset[0])
    cv_results = []
    temp = []
    totalPrecision = 0
    totalRecall = 0
    totalFScore = 0
    totalAccuracy = 0
    foldSize = int(len(dataset)/folds) #the fold size
    for i in range(0,len(dataset),foldSize):
        tempDataSet = dataset[0:i] + dataset[i+foldSize:len(dataset)] # joins to parts of the list to form the dataset to test.
        testingSet = dataset[i:i+foldSize] #takes the fold size an i to find the current test data.
        trueLabels = list(map(itemgetter(1), testingSet)) #creates a 1D array of result labels
        testingSetRemovedLabel = list(map(itemgetter(0),testingSet))
        
        #print(testingSetRemovedLabel)
        #training
        classifier = trainClassifier(tempDataSet) #classifier using 9/10th of the dataset
        #classifing
        
        results = []
        for i in testingSetRemovedLabel:
            results.append(predictLabel(i,classifier))
        
        #predictLabels using 1/10th of the dataset
        #print(results[0:10])        
        #print(trueLabels[0:10])
        
        precision = Skmet.precision_score(trueLabels, results,pos_label="fake") #finds precision score
        recall = Skmet.recall_score(trueLabels, results,pos_label="fake") #finds recall score
        f_score = 2 * (precision * recall)/(precision + recall) #calculates f_score
        accuracy = Skmet.accuracy_score(trueLabels,results) #calculate accuracy
        totalPrecision += precision
        totalRecall += recall
        if math.isnan(f_score): #if f_score not a number won't add it to the total
            totalFScore += 0
        else:
            totalFScore += f_score
        totalAccuracy += accuracy
    
    cv_results = (totalPrecision/folds,totalRecall/folds,totalFScore/folds,totalAccuracy/folds)
        
    return cv_results

#### Results